In [ ]:
from dataclasses import dataclass
from itertools import product
from pathlib import Path
from typing import Literal, Iterable
import numpy as np
import pandas as pd
from qresearch.backtest.portfolio import backtest_weights, TRADING_DAYS
from typing import Union, Sequence, Optional, Tuple
import yfinance as yf

Tickers = Union[str, Sequence[str]]

def _standardize_index(df: pd.DataFrame) -> pd.DataFrame:
    if not isinstance(df.index, pd.DatetimeIndex):
        df.index = pd.to_datetime(df.index)
    df = df.sort_index()
    # enforce tz-naive
    if df.index.tz is not None:
        df.index = df.index.tz_convert(None)
    return df


def download_ohlc_yf(
    tickers: Tickers,
    start: str,
    end: Optional[str] = None,
    *,
    auto_adjust: bool = False,
    ffill: bool = True,
    drop_all_nan_rows: bool = True,
) -> pd.DataFrame:
    """
    Download OHLC(+Adj Close, Volume) from Yahoo via yfinance.

    Behavior:
    - If tickers is a single str: returns a FLAT-column DataFrame with columns
      ["Open","High","Low","Close","Adj Close","Volume"] (subset if missing).
    - If tickers is a list/sequence: returns a MultiIndex DataFrame with columns
      (Field, Ticker) using the canonical orientation.

    Notes:
    - For indices like ^GSPC, yfinance typically provides OHLC and Adj Close.
    """
    if isinstance(tickers, str):
        tick_list = [tickers]
        single = True
    else:
        tick_list = list(tickers)
        single = False

    raw = yf.download(
        tick_list,
        start=start,
        end=end,
        auto_adjust=auto_adjust,
        progress=False,
        group_by="column",   # most stable output; can still be MultiIndex for multiple tickers
        threads=True,
    )
    if raw is None or raw.empty:
        raise ValueError("yfinance returned empty data")

    raw = _standardize_index(raw)

    # -----------------------------
    # Single ticker -> flatten
    # -----------------------------
    if single:
        if isinstance(raw.columns, pd.MultiIndex):
            # normalize to (Field, Ticker)
            lvl0 = raw.columns.get_level_values(0)
            if all(t in tick_list for t in set(lvl0)):
                raw = raw.swaplevel(0, 1, axis=1)  # (Field, Ticker)
            # slice the only ticker
            raw = raw.xs(tick_list[0], axis=1, level=1, drop_level=True)

        # now should be flat columns
        needed = {"High", "Low", "Close"}
        if not needed.issubset(set(raw.columns)):
            raise ValueError(
                f"Downloaded data missing required columns {needed}. Got: {list(raw.columns)}"
            )

        # keep canonical field ordering if present
        fields = [f for f in ["Open", "High", "Low", "Close", "Adj Close", "Volume"] if f in raw.columns]
        out = raw[fields].copy()

        out = out.replace([np.inf, -np.inf], np.nan)
        if ffill:
            out = out.ffill()
        if drop_all_nan_rows:
            out = out.dropna(how="all")
        return out

    # -----------------------------
    # Multi ticker -> MultiIndex (Field, Ticker)
    # -----------------------------
    if not isinstance(raw.columns, pd.MultiIndex):
        # defensive: yfinance sometimes returns flat if only 1 ticker; but we are in multi mode
        raise ValueError("Expected MultiIndex columns for multiple tickers, got flat columns.")

    # normalize to (Field, Ticker)
    lvl0 = raw.columns.get_level_values(0)
    if all(t in tick_list for t in set(lvl0)):
        raw = raw.swaplevel(0, 1, axis=1)

    raw = raw.sort_index(axis=1)

    # keep canonical fields if present
    fields_keep = [f for f in ["Open", "High", "Low", "Close", "Adj Close", "Volume"]
                   if f in raw.columns.get_level_values(0)]
    out = raw.loc[:, (fields_keep, tick_list)].copy()

    out = out.replace([np.inf, -np.inf], np.nan)
    if ffill:
        out = out.ffill()
    if drop_all_nan_rows:
        out = out.dropna(how="all")
    return out


def _rolling_rsrs_beta_r2(
    high: pd.Series,
    low: pd.Series,
    n: int = 18,
) -> Tuple[pd.Series, pd.Series]:
    """
    Rolling simple regression: High = alpha + beta * Low, window=n

    Returns:
      beta: rolling slope
      r2:   rolling R^2

    Implementation is vectorized using rolling moments:
      beta = cov(Low, High) / var(Low)
      R^2  = corr(Low, High)^2 = cov^2 / (var(Low)*var(High))   (for simple regression with intercept)

    Notes:
    - Uses ddof=0 consistency with your z-score std(ddof=0).
    - Handles divide-by-zero by returning NaN where not defined.
    """
    high = pd.Series(high).astype(float)
    low = pd.Series(low).astype(float)
    idx = high.index.intersection(low.index)
    high = high.reindex(idx)
    low = low.reindex(idx)

    # rolling means
    mx = low.rolling(n).mean()
    my = high.rolling(n).mean()

    # rolling second moments
    mxx = (low * low).rolling(n).mean()
    myy = (high * high).rolling(n).mean()
    mxy = (low * high).rolling(n).mean()

    varx = mxx - mx * mx
    vary = myy - my * my
    cov = mxy - mx * my

    # beta
    with np.errstate(divide="ignore", invalid="ignore"):
        beta = cov / varx

    # r2 = corr^2
    with np.errstate(divide="ignore", invalid="ignore"):
        r2 = (cov * cov) / (varx * vary)

    beta = beta.replace([np.inf, -np.inf], np.nan)
    r2 = r2.replace([np.inf, -np.inf], np.nan)

    # Clip r2 to [0,1] where numerical noise occurs
    r2 = r2.clip(lower=0.0, upper=1.0)

    beta.name = "beta"
    r2.name = "r2"
    return beta, r2


In [ ]:
# =========================================================
# 0) Assumptions (you already have these in your project)
# =========================================================
# - download_ohlc_yf(ticker, start, end, auto_adjust=False) -> DataFrame with columns:
#   ["Open","High","Low","Close","Adj Close","Volume"] (some may be missing)
# - _rolling_rsrs_beta_r2(high: Series, low: Series, n: int) -> (beta Series, r2 Series)
#
# If your function name differs (e.g., download_ohlc), just rename below.

# =========================================================
# 1) RSRS definitions
# =========================================================
RSRSVariant = Literal["z", "z_adj", "z_right"]

@dataclass(frozen=True)
class RSRSParams:
    n: int
    m: int
    s: float
    variant: RSRSVariant
    ma_len: int  # 0 => disable MA filter (price_filter=False)

def rsrs_indicator(
    ohlc: pd.DataFrame,
    n: int = 18,
    m: int = 600,
    variant: RSRSVariant = "z_right",
) -> pd.Series:
    """
    Compute RSRS indicator series on the same date index as ohlc.
    - n: regression window for beta and r2
    - m: standardization window for beta z-score
    - variant:
        z       = zscore(beta)
        z_adj   = z * r2
        z_right = (z * r2) * beta
    """
    beta, r2 = _rolling_rsrs_beta_r2(high=ohlc["High"], low=ohlc["Low"], n=n)

    mu = beta.rolling(m).mean()
    sd = beta.rolling(m).std(ddof=0)
    z = (beta - mu) / sd

    z_adj = z * r2
    z_right = z_adj * beta

    return {"z": z, "z_adj": z_adj, "z_right": z_right}[variant]

def rsrs_weights_from_score(
    score: pd.Series,
    close: pd.Series,
    trade_ticker: str,
    s: float,
    ma_len: int = 20,
    ma_compare_lag1: int = 1,
    ma_compare_lag2: int = 3,
) -> pd.DataFrame:
    """
    LONG-ONLY in/out timing weights for a single traded ticker, using precomputed score.
    Decision-time weights (as-of close t), UN-SHIFTED; backtest_weights() shifts by 1.

    Hysteresis rules:
      - if score_t < -s: weight = 0
      - else if score_t >  s: weight = 1, with optional MA filter
      - else: hold previous weight

    MA filter enabled iff ma_len > 0:
      MA(t-lag1) > MA(t-lag2)
    """
    idx = score.index
    close = close.reindex(idx).astype(float)

    if ma_len and ma_len > 0:
        ma = close.rolling(ma_len).mean()
        ma_ok = ma.shift(ma_compare_lag1) > ma.shift(ma_compare_lag2)
    else:
        ma_ok = pd.Series(True, index=idx)

    w = pd.Series(0.0, index=idx, dtype=float)
    state = 0.0

    # Use .to_numpy() for speed
    sc = score.to_numpy()
    ok = ma_ok.to_numpy()

    for i in range(len(w)):
        it = sc[i]
        if not np.isfinite(it):
            w.iat[i] = state
            continue

        if it < -s:
            state = 0.0
        elif it > s:
            if bool(ok[i]):
                state = 1.0
            # else: hold state
        # else: hold state

        w.iat[i] = state

    return pd.DataFrame({trade_ticker: w}, index=idx)

# =========================================================
# 2) Performance stats (same as your style)
# =========================================================
def perf_stats(returns: pd.Series, trading_days: int = TRADING_DAYS) -> dict[str, float]:
    r = returns.fillna(0.0)
    eq = (1.0 + r).cumprod()
    dd = eq / eq.cummax() - 1.0

    ann_ret = (eq.iat[-1] ** (trading_days / max(1, len(eq))) - 1.0) if len(eq) > 1 else 0.0
    ann_vol = float(r.std(ddof=0) * np.sqrt(trading_days)) if len(r) > 1 else 0.0
    sharpe = (ann_ret / ann_vol) if ann_vol > 1e-12 else np.nan
    max_dd = float(dd.min()) if len(dd) else 0.0
    return {
        "ann_ret": float(ann_ret),
        "ann_vol": float(ann_vol),
        "sharpe": float(sharpe),
        "max_dd": float(max_dd),
        "total_return": float(eq.iat[-1] - 1.0) if len(eq) else 0.0,
    }

# =========================================================
# 3) Walk-forward window generator
# =========================================================
def walkforward_year_windows(
    index: pd.DatetimeIndex,
    train_years: int = 5,
    test_years: int = 1,
    step_years: int = 1,
) -> list[tuple[pd.Timestamp, pd.Timestamp, pd.Timestamp, pd.Timestamp]]:
    """
    Returns list of (train_start, train_end, test_start, test_end), inclusive ends.
    """
    idx = pd.DatetimeIndex(index).sort_values()
    start = idx.min()
    end = idx.max()

    windows = []
    cur_train_start = start

    while True:
        train_end = cur_train_start + pd.DateOffset(years=train_years) - pd.DateOffset(days=1)
        test_start = train_end + pd.DateOffset(days=1)
        test_end = test_start + pd.DateOffset(years=test_years) - pd.DateOffset(days=1)

        if test_end > end:
            break

        # Snap to available trading dates (next/prev)
        ts = idx[idx >= cur_train_start]
        if len(ts) == 0:
            break
        train_start2 = ts[0]

        te = idx[idx <= train_end]
        if len(te) == 0:
            break
        train_end2 = te[-1]

        tss = idx[idx >= test_start]
        if len(tss) == 0:
            break
        test_start2 = tss[0]

        tee = idx[idx <= test_end]
        if len(tee) == 0:
            break
        test_end2 = tee[-1]

        windows.append((train_start2, train_end2, test_start2, test_end2))

        # step forward
        cur_train_start = cur_train_start + pd.DateOffset(years=step_years)

    return windows

# =========================================================
# 4) Grid (FAST) — keep small to avoid long runtime
# =========================================================
def make_fast_grid() -> list[RSRSParams]:
    # Recommended fast grid
    variants: list[RSRSVariant] = ["z_right"]  # add "z_adj" later if you want
    ns = [16, 18]
    ms = [250, 300, 600]
    ss = [0.5, 0.6, 0.7, 0.8]
    mas = [0, 8, 20]  # 0 => disable MA filter

    out: list[RSRSParams] = []
    for n, m, s, v, ma in product(ns, ms, ss, variants, mas):
        out.append(RSRSParams(n=n, m=m, s=s, variant=v, ma_len=ma))
    return out

# =========================================================
# 5) Core runner: one ticker, walk-forward OOS, select max Sharpe on train
# =========================================================
def run_rsrs_oos_one_ticker(
    trade_ticker: str,
    start: str,
    end: str,
    grid: Iterable[RSRSParams],
    *,
    train_years: int = 5,
    test_years: int = 1,
    step_years: int = 1,
    fee_bps: float = 0.0,
    rf_annual: float = 0.0,
    long_only: bool = True,
    allow_leverage: bool = False,
    fill_weights: Literal["zero", "ffill", "error"] = "zero",
) -> tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """
    Returns:
      - wf_rows: per-window selection rows
      - param_summary: grouped by params across OOS windows
      - oos_summary: single-row summary for the concatenated OOS equity
    """
    # ---- download OHLC ----
    ohlc = download_ohlc_yf(trade_ticker, start=start, end=end, auto_adjust=False)
    ohlc = ohlc.sort_index()

    # tradable price series
    px_col = "Adj Close" if "Adj Close" in ohlc.columns else "Close"
    px = ohlc[px_col].astype(float)
    prices = pd.DataFrame({trade_ticker: px}, index=ohlc.index)

    # precompute scores + weights once per param (big speed-up)
    close = ohlc["Close"].astype(float)
    weights_cache: dict[RSRSParams, pd.DataFrame] = {}

    for p in grid:
        score = rsrs_indicator(ohlc=ohlc, n=p.n, m=p.m, variant=p.variant)
        w = rsrs_weights_from_score(
            score=score,
            close=close,
            trade_ticker=trade_ticker,
            s=p.s,
            ma_len=p.ma_len,
            ma_compare_lag1=1,
            ma_compare_lag2=3,
        )
        weights_cache[p] = w

    # walk-forward windows
    windows = walkforward_year_windows(
        ohlc.index, train_years=train_years, test_years=test_years, step_years=step_years
    )

    wf_rows = []
    oos_net_rets = []

    for (train_start, train_end, test_start, test_end) in windows:
        # ---- select best params on TRAIN (max Sharpe) ----
        best_p = None
        best_metric = -np.inf

        for p in grid:
            w_full = weights_cache[p]
            pr_tr = prices.loc[train_start:train_end]
            w_tr = w_full.loc[train_start:train_end]

            if len(pr_tr) < 5 or len(w_tr) < 5:
                continue

            res_tr = backtest_weights(
                prices=pr_tr,
                weights=w_tr,
                fee_bps=fee_bps,
                rf_annual=rf_annual,
                long_only=long_only,
                allow_leverage=allow_leverage,
                fill_weights=fill_weights,
            )
            st_tr = perf_stats(res_tr.net_ret)
            metric = st_tr["sharpe"]

            if np.isfinite(metric) and metric > best_metric:
                best_metric = metric
                best_p = p

        if best_p is None:
            # no valid selection; skip this window
            continue

        # ---- evaluate best params on TEST ----
        w_full = weights_cache[best_p]
        pr_te = prices.loc[test_start:test_end]
        w_te = w_full.loc[test_start:test_end]

        res_te = backtest_weights(
            prices=pr_te,
            weights=w_te,
            fee_bps=fee_bps,
            rf_annual=rf_annual,
            long_only=long_only,
            allow_leverage=allow_leverage,
            fill_weights=fill_weights,
        )
        st_te = perf_stats(res_te.net_ret)

        wf_rows.append({
            "ticker": trade_ticker,
            "train_start": train_start,
            "train_end": train_end,
            "test_start": test_start,
            "test_end": test_end,
            "sel_metric_train": float(best_metric),
            "n": best_p.n,
            "m": best_p.m,
            "s": best_p.s,
            "variant": best_p.variant,
            "ma_len": best_p.ma_len,
            "oos_ann_ret": st_te["ann_ret"],
            "oos_ann_vol": st_te["ann_vol"],
            "oos_sharpe": st_te["sharpe"],
            "oos_max_dd": st_te["max_dd"],
            "oos_total_return": st_te["total_return"],
            "oos_avg_exposure": float(res_te.exposure.mean()) if len(res_te.exposure) else 0.0,
            "oos_turnover": float(res_te.turnover.sum()) if len(res_te.turnover) else 0.0,
        })

        oos_net_rets.append(res_te.net_ret.rename(test_start))

    wf_df = pd.DataFrame(wf_rows)
    if wf_df.empty:
        return wf_df, pd.DataFrame(), pd.DataFrame()

    # ---- param summary across OOS windows ----
    gcols = ["n", "m", "s", "variant", "ma_len"]
    param_summary = (
        wf_df.groupby(gcols)
        .agg(
            mean_oos_sharpe=("oos_sharpe", "mean"),
            med_oos_sharpe=("oos_sharpe", "median"),
            mean_oos_ann_ret=("oos_ann_ret", "mean"),
            mean_oos_max_dd=("oos_max_dd", "mean"),
            n_windows=("oos_sharpe", "count"),
        )
        .sort_values("mean_oos_sharpe", ascending=False)
        .reset_index()
    )

    # ---- concatenated OOS equity (stitch test windows) ----
    # concatenate net returns in chronological order (they are separate test windows)
    oos_all = pd.concat([s for s in oos_net_rets], axis=0).sort_index()
    oos_summary = pd.DataFrame([{
        "ticker": trade_ticker,
        **{f"oos_{k}": v for k, v in perf_stats(oos_all).items()},
    }])

    return wf_df, param_summary, oos_summary

# =========================================================
# 6) Multi-market runner + saving outputs
# =========================================================
def run_multi_market_rsrs_oos(
    tickers: list[str],
    *,
    start: str,
    end: str,
    out_dir: str = "./rsrs_oos_runs",
    train_years: int = 5,
    test_years: int = 1,
    step_years: int = 1,
    fee_bps: float = 0.0,
    rf_annual: float = 0.0,
) -> None:
    grid = make_fast_grid()

    out_path = Path(out_dir)
    out_path.mkdir(parents=True, exist_ok=True)

    all_wf = []
    all_param = []
    all_oos = []

    for t in tickers:
        wf_df, param_df, oos_df = run_rsrs_oos_one_ticker(
            trade_ticker=t,
            start=start,
            end=end,
            grid=grid,
            train_years=train_years,
            test_years=test_years,
            step_years=step_years,
            fee_bps=fee_bps,
            rf_annual=rf_annual,
        )

        if not wf_df.empty:
            all_wf.append(wf_df)
        if not param_df.empty:
            param_df = param_df.copy()
            param_df.insert(0, "ticker", t)
            all_param.append(param_df)
        if not oos_df.empty:
            all_oos.append(oos_df)

        # per-ticker save (so you never lose partial work)
        stamp = pd.Timestamp.utcnow().strftime("%Y%m%d_%H%M%S")
        wf_df.to_csv(out_path / f"{t}_walkforward_{stamp}.csv", index=False)
        param_df.to_csv(out_path / f"{t}_param_summary_{stamp}.csv", index=False)
        oos_df.to_csv(out_path / f"{t}_oos_summary_{stamp}.csv", index=False)

    # combined save
    stamp = pd.Timestamp.utcnow().strftime("%Y%m%d_%H%M%S")

    if all_wf:
        wf_all = pd.concat(all_wf, ignore_index=True)
        wf_all.to_csv(out_path / f"ALL_walkforward_{stamp}.csv", index=False)

    if all_param:
        param_all = pd.concat(all_param, ignore_index=True)
        param_all.to_csv(out_path / f"ALL_param_summary_{stamp}.csv", index=False)

    if all_oos:
        oos_all = pd.concat(all_oos, ignore_index=True)
        oos_all.to_csv(out_path / f"ALL_oos_summary_{stamp}.csv", index=False)

    print(f"Saved results to: {out_path.resolve()}")

In [ ]:
import itertools

def make_param_grid(
    n_list=(12, 16, 18),
    m_list=(250, 300, 600),
    s_list=(0.5, 0.6, 0.7),
    variants=("z_right",),          # keep fast; optionally add "z_adj"
    ma_lens=(0, 5, 10, 20),
) -> list[RSRSParams]:
    return [
        RSRSParams(n=n, m=m, s=s, variant=v, ma_len=ma)
        for n, m, s, v, ma in itertools.product(n_list, m_list, s_list, variants, ma_lens)
    ]


def year_windows(index: pd.DatetimeIndex, train_years: int = 5, test_years: int = 1) -> list[tuple[pd.Timestamp, pd.Timestamp, pd.Timestamp, pd.Timestamp]]:
    """
    Expanding-start, rolling-end style similar to your earlier prints:
    - Train: from first date up to year-end of (start_year + train_years - 1)
    - Test: next year(s)
    """
    idx = pd.DatetimeIndex(index).sort_values()
    start_date = idx.min()
    start_year = start_date.year
    last_year = idx.max().year

    windows = []
    train_start = start_date

    for test_start_year in range(start_year + train_years, last_year - test_years + 1):
        train_end = pd.Timestamp(year=test_start_year - 1, month=12, day=31)
        test_start = pd.Timestamp(year=test_start_year, month=1, day=1)
        test_end = pd.Timestamp(year=test_start_year + test_years - 1, month=12, day=31)

        # snap to available trading days by slicing later (no need perfect day alignment here)
        windows.append((train_start, train_end, test_start, test_end))

    return windows


def rsrs_timing_weights(
    ohlc: pd.DataFrame,
    trade_ticker: str,
    n: int = 16,
    m: int = 300,
    s: float = 0.7,
    variant: Literal["z", "z_adj", "z_right"] = "z_right",
    ma_len: int = 20,                 # set 0 to disable MA filter
    ma_compare_lag1: int = 1,
    ma_compare_lag2: int = 3,
) -> pd.DataFrame:
    """
    Decision-time weights (as-of close t), UN-SHIFTED.
    backtest_weights() will apply shift(1) to enforce no-lookahead.

    Hysteresis:
      - score < -s  => state = 0
      - score > +s  => state = 1 IF (MA filter ok or disabled)
      - else hold previous
    """
    score = rsrs_indicator(ohlc=ohlc, n=n, m=m, variant=variant)
    close = ohlc["Close"].astype(float)

    if ma_len is None or ma_len <= 0:
        ma_ok = pd.Series(True, index=ohlc.index)
    else:
        ma = close.rolling(ma_len).mean()
        ma_ok = ma.shift(ma_compare_lag1) > ma.shift(ma_compare_lag2)

    w = pd.Series(0.0, index=ohlc.index, dtype=float)
    state = 0.0

    for i in range(len(w)):
        sc = score.iat[i]
        ok = bool(ma_ok.iat[i])

        if not np.isfinite(sc):
            w.iat[i] = state
            continue

        if sc < -s:
            state = 0.0
        elif sc > s:
            if ok:
                state = 1.0
            # else hold
        # else hold

        w.iat[i] = state

    return pd.DataFrame({trade_ticker: w})


def walkforward_oos_optimize_one_ticker(
    ticker: str,
    ohlc: pd.DataFrame,
    start: str,
    end: str,
    grid: list[RSRSParams],
    fee_bps: float = 0.0,
    rf_annual: float = 0.0,
    train_years: int = 5,
    test_years: int = 1,
    select_metric: str = "sharpe",
) -> tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """
    Returns:
      - oos_summary (1 row)
      - param_summary (grouped stats per param set)
      - walkforward_detail (per window selection + OOS stats)
    """
    # tradable price
    price_col = "Adj Close" if "Adj Close" in ohlc.columns else "Close"
    px = ohlc[price_col].astype(float)
    prices = pd.DataFrame({ticker: px}, index=ohlc.index)

    # cache: per param => full-sample backtest result
    cache = {}
    for p in grid:
        w = rsrs_timing_weights(
            ohlc=ohlc,
            trade_ticker=ticker,
            n=p.n, m=p.m, s=p.s,
            variant=p.variant,
            ma_len=p.ma_len,
            ma_compare_lag1=1,
            ma_compare_lag2=3,
        )
        res = backtest_weights(
            prices=prices,
            weights=w,
            fee_bps=fee_bps,
            rf_annual=rf_annual,
            long_only=True,
            allow_leverage=False,
            fill_weights="zero",
        )
        cache[p] = (w, res)

    # windows
    wins = year_windows(prices.index, train_years=train_years, test_years=test_years)

    wf_rows = []
    oos_concat = []

    for (train_start, train_end, test_start, test_end) in wins:
        # slice masks
        train_mask = (prices.index >= train_start) & (prices.index <= train_end)
        test_mask = (prices.index >= test_start) & (prices.index <= test_end)

        # skip empty test slices
        if test_mask.sum() < 10:
            continue

        # select best param on train
        best_p = None
        best_val = -np.inf

        for p in grid:
            _, res = cache[p]
            r_train = res.net_ret.loc[train_mask]

            st = perf_stats(r_train)
            val = st.get(select_metric, np.nan)
            if np.isfinite(val) and val > best_val:
                best_val = val
                best_p = p

        if best_p is None:
            continue

        # evaluate OOS on test
        _, res_best = cache[best_p]
        r_test = res_best.net_ret.loc[test_mask]
        st_oos = perf_stats(r_test)

        oos_concat.append(r_test)

        wf_rows.append({
            "ticker": ticker,
            "train_start": str(pd.Timestamp(train_start).date()),
            "train_end": str(pd.Timestamp(train_end).date()),
            "test_start": str(pd.Timestamp(test_start).date()),
            "test_end": str(pd.Timestamp(test_end).date()),
            "sel_metric_train": float(best_val),
            "n": best_p.n,
            "m": best_p.m,
            "s": best_p.s,
            "variant": best_p.variant,
            "ma_len": best_p.ma_len,
            "oos_ann_ret": st_oos["ann_ret"],
            "oos_ann_vol": st_oos["ann_vol"],
            "oos_sharpe": st_oos["sharpe"],
            "oos_max_dd": st_oos["max_dd"],
            "oos_total_return": st_oos["total_return"],
            "oos_avg_exposure": float(res_best.exposure.loc[test_mask].mean()),
            "oos_turnover": float(res_best.turnover.loc[test_mask].sum()),
        })

    walkforward_detail = pd.DataFrame(wf_rows)

    # OOS concatenated summary
    if len(oos_concat) == 0:
        oos_summary = pd.DataFrame([{
            "ticker": ticker,
            "oos_ann_ret": 0.0,
            "oos_ann_vol": 0.0,
            "oos_sharpe": np.nan,
            "oos_max_dd": 0.0,
            "oos_total_return": 0.0,
        }])
    else:
        oos_all = pd.concat(oos_concat).sort_index()
        st_all = perf_stats(oos_all)
        oos_summary = pd.DataFrame([{
            "ticker": ticker,
            "oos_ann_ret": st_all["ann_ret"],
            "oos_ann_vol": st_all["ann_vol"],
            "oos_sharpe": st_all["sharpe"],
            "oos_max_dd": st_all["max_dd"],
            "oos_total_return": st_all["total_return"],
        }])

    # Param summary: average OOS Sharpe across windows where selected
    if walkforward_detail.empty:
        param_summary = pd.DataFrame(columns=[
            "ticker","n","m","s","variant","ma_len",
            "mean_oos_sharpe","med_oos_sharpe","mean_oos_ann_ret","mean_oos_max_dd","n_windows"
        ])
    else:
        g = walkforward_detail.groupby(["ticker","n","m","s","variant","ma_len"])
        param_summary = g.agg(
            mean_oos_sharpe=("oos_sharpe","mean"),
            med_oos_sharpe=("oos_sharpe","median"),
            mean_oos_ann_ret=("oos_ann_ret","mean"),
            mean_oos_max_dd=("oos_max_dd","mean"),
            n_windows=("oos_sharpe","count"),
        ).reset_index().sort_values(["ticker","mean_oos_sharpe"], ascending=[True, False])

    return oos_summary, param_summary, walkforward_detail

In [ ]:
from datetime import datetime

tickers = ["^HSI", "^GSPC", "^N225", "^STOXX"]  # add/remove as needed
start = "2000-01-01"
end = "2025-12-31"

fee_bps = 0.0
rf_annual = 0.0

train_years = 5
test_years = 1

grid = make_param_grid(
    n_list=(12, 16, 18),
    m_list=(250, 300, 600),
    s_list=(0.5, 0.6, 0.7),
    variants=("z_right",),          # keep fast; add "z_adj" if you want
    ma_lens=(0, 5, 10, 20),
)

all_oos = []
all_param = []
all_wf = []

for tk in tickers:
    print(f"\n=== Running {tk} ===")
    ohlc = download_ohlc_yf(tk, start=start, end=end, auto_adjust=False)

    oos_summary, param_summary, walkforward_detail = walkforward_oos_optimize_one_ticker(
        ticker=tk,
        ohlc=ohlc,
        start=start,
        end=end,
        grid=grid,
        fee_bps=fee_bps,
        rf_annual=rf_annual,
        train_years=train_years,
        test_years=test_years,
        select_metric="sharpe",
    )

    all_oos.append(oos_summary)
    all_param.append(param_summary)
    all_wf.append(walkforward_detail)

ALL_oos_summary = pd.concat(all_oos, ignore_index=True)
ALL_param_summary = pd.concat(all_param, ignore_index=True)
ALL_walkforward = pd.concat(all_wf, ignore_index=True)

ts = datetime.now().strftime("%Y%m%d_%H%M%S")
ALL_oos_summary.to_csv(f"ALL_oos_summary_{ts}.csv", index=False)
ALL_param_summary.to_csv(f"ALL_param_summary_{ts}.csv", index=False)
ALL_walkforward.to_csv(f"ALL_walkforward_{ts}.csv", index=False)

print("\nSaved:")
print(f"  ALL_oos_summary_{ts}.csv")
print(f"  ALL_param_summary_{ts}.csv")
print(f"  ALL_walkforward_{ts}.csv")

print("\n=== ALL OOS Summary ===")
print(ALL_oos_summary.sort_values("oos_sharpe", ascending=False).to_string(index=False))